# Multi-Agent Research & Summarization Workflow

# Technologies: Python, LangGraph, OpenAI GPT-3.5, TypedDict
Developed a multi-agent AI workflow with LangGraph and GPT-3.5, enabling automated research, summarization, and stateful coordination between agents.

In [ ]:
import os
from langchain_openai import OpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict
from getpass import getpass # Import getpass

# Set up the OpenAI model
# Retrieve the API key directly in this cell
openai_api_key = getpass("Enter your OpenAI API Key: ")
llm = OpenAI(model="gpt-3.5-turbo-instruct", api_key=openai_api_key)


# Define a state to pass between agents
class ResearchState(TypedDict):
    research_data: str
    summary: str

# Agent functions
def research_agent(state: ResearchState) -> ResearchState:
    prompt = "Research eco-friendly products and provide 2-3 sentences about them."
    state["research_data"] = llm.invoke(prompt) # Use invoke() instead of __call__() for newer LangChain versions
    return state

def summarize_agent(state: ResearchState) -> ResearchState:
    prompt = f"Summarize this in one sentence: {state['research_data']}"
    state["summary"] = llm.invoke(prompt) # Use invoke() instead of __call__() for newer LangChain versions
    return state

# Build the workflow graph
workflow = StateGraph(ResearchState)
workflow.add_node("researcher", research_agent)
workflow.add_node("summarizer", summarize_agent)
workflow.add_edge("researcher", "summarizer")
workflow.add_edge("summarizer", END)
workflow.set_entry_point("researcher")

# Compile and run
app = workflow.compile()
initial_state = {"research_data": "", "summary": ""}
result = app.invoke(initial_state)

# Show results
print("Research Data:", result["research_data"])
print("Summary:", result["summary"])